# Observed discharge
***

__Author__: Chus Casado<br>
__Date__:   08-06-2022<br>

__Introduction__:<br>


In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
import xarray as xr

In [ ]:
import os
path_root = os.getcwd()

In [ ]:
os.chdir('../py/')
from convert import dict2da
os.chdir(path_root)

## Observed data 

In [ ]:
stations = pd.read_parquet('../results/reporting_points/reporting_points_over_500km2.parquet')
stations = stations[['name', 'X', 'Y', 'area', 'subcatchment', 'river', 'catchment', 'country', 'n_events_obs', 'rl5', 'rl20', 'rl100']]
stations.head()

In [ ]:
stations.loc[stations.catchment == 'Ebro'].sort_values('n_events_obs', ascending=False)

In [ ]:
stations.to_csv('../results/reporting_points/reporting_points_over_500km2.csv')

In [ ]:
stations = pd.read_csv('../data/discharge/observed/6hourData.csv', index_col='station_id')

In [ ]:
data = pd.read_csv('../data/discharge/observed/output-6h-nrt-operational.txt', index_col='Station ID', na_values=[-9999, -999, -99])
data.Timestamp = pd.to_datetime(data.Timestamp)

In [ ]:
stns = data.index.unique().tolist()
stns.sort()
dates = pd.date_range(datetime(2018, 1, 1), data.Timestamp.max() + timedelta(hours=1), freq='6h')
obs = pd.DataFrame(index=dates, columns=stns, dtype=float)
for i, stn in enumerate(stns):
    
    print(f'{i+1:02d} from {len(stns):02d}', end='\r')
    
    aux = data.loc[stn, ['Timestamp', 'AvgValue']]
    aux.set_index('Timestamp', drop=True, inplace=True)
    aux = aux.loc['2018-01-01':, 'AvgValue']
    aux[(aux < 0) | (aux > 10000)] = np.nan
    obs.loc[aux.index, stn] = aux.values

## Reanalysis data

In [ ]:
def KGE(observado, simulado, sa=1, sb=1, sr=1):
    """Calcula el coeficiente de eficiencia de Kling-Gupta.
    
    Parámetros:
    -----------
    observado:   series. Serie observada
    simulado:    series. Serie simulada
    sa, sb, sr: integer. Factores de escala de los tres términos del KGE: alpha, beta y coeficiente de correlación, respectivamente
    
    Salida:
    -------
    KGE:        float. Eficienica de Kling-Gupta"""
    
    # Eliminar pasos sin dato
    data = pd.concat((observado, simulado), axis=1)
    data.columns = ['obs', 'sim']
    data.dropna(axis=0, how='any', inplace=True)
    # Para la función si no hay datos
    if data.shape[0] == 0:
        return

    # calcular cada uno de los términos del KGE
    alpha = data.sim.std() / data.obs.std()
    beta = data.sim.mean() / data.obs.mean()
    r = np.corrcoef(data.obs, data.sim)[0, 1]
    
    # Cacular KGE
    ED = np.sqrt((sr * (r - 1))**2 + (sa * (alpha - 1))**2 + (sb * (beta - 1))**2)
    KGE = 1 - ED
    
    return KGE

In [ ]:
rea = pd.DataFrame(dtype=float)
files = glob.glob('../data/discharge/reanalysis/*.csv')
for file in files:
    aux = pd.read_csv(file, parse_dates=True, index_col=0)
    aux.columns = aux.columns.astype(int)
    rea = pd.concat((rea, aux), axis=0)

## Comparison

In [ ]:
# cut series so that they have the same extent
start = max(obs.index.min(), rea.index.min())
end = min(obs.index.max(), rea.index.max())
obs = obs.loc[start:end,:]
rea = rea.loc[start:end,:]

In [ ]:
kge = pd.Series(index=stns, dtype=float)
for stn in stns:
    kge[stn] = KGE(obs[stn], rea[stn])

In [ ]:
kge.sort_values()

In [ ]:
stn = kge.idxmin()
print(stn)

In [ ]:
stn = '2996'

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(obs.index, obs[stn], label='obs')
ax.plot(rea.index, rea[stn], label='sim')
ax.legend();

stations.loc[stn]

In [ ]:
rea.plot(figsize=(16, 5));

***

Analyse the availability of data in the Hydro Database.

**Historical**

In [ ]:
historic_operational = pd.read_excel('../data/discharge/observed/DB/OP_hdata_ranges_EHDCC_8029.xlsx', sheet_name='6h', header=0, index_col=0)
historic = pd.read_csv('../data/discharge/observed/DB/hdata_ranges_EHDCC_8029_202305151428_EFAS3_Hydro_ro.csv', sep=';', parse_dates=True, index_col='ID')

print('no. points with historic operational data:\t{0}'.format(historic_operational.shape[0]))
print('no. points with historic data:\t\t\t{0}'.format(historic.shape[0]))

**Real time**

In [ ]:
real_time_operational = pd.read_excel('../data/discharge/observed/DB/OP_rtdata_ranges_EHDCC_8029.xlsx', sheet_name='6h', header=0, index_col=0)
real_time = pd.read_csv('../data/discharge/observed/DB/rtdata_ranges_EHDCC_8029_202305151623_EFAS3_Hydro_ro.csv', sep=';', parse_dates=True, index_col='ID')

print('no. points with real time operational data:\t{0}'.format(real_time_operational.shape[0]))
print('no. points with real time data:\t\t\t{0}'.format(real_time.shape[0]))

In [ ]:
print('no. stations in both historic and real time data:', len(set(historic.index).intersection(real_time.index)))
print('no. stations in both historic operational and real time data operational:', len(set(historic_operational.index).intersection(real_time_operational.index)))

In [ ]:
stns = [2996,
 1569,
 1469,
 1454,
 1688,
 1572,
 4292,
 2390,
 2275,
 2999,
 4324,
 2388,
 4001,
 1573,
 2951,
 3018,
 2707,
 1680,
 3166,
 1580,
 4627,
 310,
 4174,
 187,
 223,
 1568,
 2701,
 1194,
 3017,
 4114,
 2997,
 4490,
 4015,
 4500,
 4013,
 4504,
 4578,
 2685,
 119,
 2596,
 2428,
 166,
 167,
 195,
 247,
 578,
 581,
 954,
 1410,
 1411,
 1529,
 1652,
 1653,
 1737,
 1738,
 2016,
 165,
 2149,
 2341,
 2373,
 2148]

In [ ]:
for stn in stns:
    if stn in historic.index:
        print(stn, 'historic', *historic.loc[stn, ['start', 'end']], sep='\t')
    if stn in historic_operational.index:
        print(stn, 'historic op.', *historic_operational.loc[stn, ['start', 'end']], sep='\t')
    if stn in real_time.index:
        print(stn, 'real time', *real_time.loc[stn, ['start', 'end']], sep='\t')
    if stn in real_time_operational.index:
        print(stn, 'real time op.', *real_time_operational.loc[stn, ['start', 'end']], sep='\t')
    print()

In [ ]:
path_hist_op = '../data/discharge/observed/historic_operational/'
files_obs = glob.glob(f'{path_hist_op}*.csv')
stns = [int(file.split('\\')[-1].split('_')[0]) for file in files_obs]

Qobs = pd.DataFrame(index=pd.date_range(datetime(2020, 1, 1), datetime(2023, 1, 1), freq='6h'),
                    columns=stns,
                    dtype=float)
for file, stn in zip(files_obs, stns):
    raw = pd.read_csv(file, parse_dates=True, index_col=0)
    raw.drop_duplicates(keep='first', inplace=True)
    raw = raw.loc[raw.Variable == 'D', 'Value']
    Qobs.loc[raw.index, stn] = raw
    
Qobs = Qobs.loc[Qobs.first_valid_index():Qobs.last_valid_index()]
print(Qobs.shape)
Qobs.head()

<font color='red'>**Negative discharge!!!**</font>

In [ ]:
path_rean = '../data/discharge/reanalysis/'

In [ ]:
xr.open_dataarray(f'{path_rean}{954:04}.nc')

In [ ]:
Qrean = dict2da({stn: xr.open_dataarray(f'{path_rean}{stn:04}.nc') for stn in Qobs.columns}, dim='id').transpose()
Qrean = Qrean.sel(datetime=slice(Qobs.index.min(), Qobs.index.max()))
Qrean = Qrean.to_pandas()

In [ ]:
lw = .7
alpha = .1

fig, axes = plt.subplots(nrows=len(stns), figsize=(16, 3 * len(stns)), sharex=True)

start, end = Qobs.index.min(), Qobs.index.max()

for ax, stn in zip(axes, stns):
    ax.plot(Qobs[stn], c='dimgray', lw=lw, label='observed')
    ax.plot(Qrean[stn], c='steelblue', lw=lw, label='reanalysis')
    ax.fill_between([start, end], *stations.loc[stn, ['rl5', 'rl20']], alpha=alpha, color='red', edgecolor=None)
    ax.fill_between([start, end], *stations.loc[stn, ['rl20', 'rl100']], alpha=alpha, color='purple', edgecolor=None)
    ax.set_title('{0} - River {1} ({2})'.format(stn, *stations.loc[stn, ['river', 'catchment']]))
    ax.set_ylabel('discharge (m3/s)')
    kge = KGE(Qobs[stn], Qrean[stn])
    ax.text(.005, .92, f'KGE = {kge:.2f}', transform=ax.transAxes)
ax.set_xlim(start, end);